In [1]:
import open3d as otd
from tqdm import tqdm
import torch as tc
import numpy as np
import pandas as pd
from curve_utils import CurveUtils
from curve_generator import CurveGenerator

device = tc.device("cuda" if tc.cuda.is_available() else "cpu")

In [2]:
their_semantics = [
    'Neck girth',
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Upper arm girth R',
    'Thigh girth R',
    'Height (m)',
    'Total crotch length',
    'Across back shoulder width',
]
our_semantic = [
    'neck_girth', # 5.3.2
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'upper_arm_girth', # 5.3.16
    'thigh_girth', # 5.3.20
    'stature', # 5.1.1
    'crotch_length', # 5.1.15
    'shoulder_width', # 5.4.3
]
curve_index = {
    'neck_girth':4, # 5.3.2
    'bust_chest_girth': 0, # 5.3.4
    'waist_girth': 1, # 5.3.10
    'hip_girth': 1, # 5.3.13
    'upper_arm_girth': 3, # 5.3.16
    'thigh_girth': 2, # 5.3.20
    'stature': 0,
    'crotch_length': 5
}
curves_names = list(curve_index.keys())

In [3]:
selected_subjects = dict()
selected_measures = dict()

for gender in ['female', 'male']:
    measures = pd.read_pickle(f'data/{gender}_measures.zip')
    measures.index = measures['Subject']
    measures = measures[measures['Measuring station'] == "MOVE4D"]
    temp_measures = measures[their_semantics].iloc[::2]
    
    mean_measures = measures._get_numeric_data().mean(axis=0)
    selected_subjects[gender] = 'IEEEP2_07' if gender == 'female' else 'IEEEP2_04'
    selected_measures[gender] = temp_measures.loc[selected_subjects[gender]]
    selected_subjects[gender] = temp_measures.index.get_loc(selected_subjects[gender])*2
    selected_measures[gender].index = our_semantic
    selected_measures[gender]['stature'] *= 1000

In [4]:
mesh = otd.io.read_triangle_mesh("tests/female_4_body.obj")
mesh.compute_vertex_normals()

TriangleMesh with 6890 points and 13776 triangles.

In [5]:
for nsubs in range(4):
    temp_mesh = mesh.subdivide_loop(number_of_iterations=nsubs)
    print(temp_mesh)
    body = tc.FloatTensor(np.asarray(temp_mesh.vertices)).to(device)
    template = tc.LongTensor(np.asarray(temp_mesh.triangles)).to(device)
    measures = selected_measures['female']
    result = CurveGenerator.get_curves(body, selected_measures['female'], template, device)
    best_curves, data = CurveUtils.select_better(result, body, measures, device)
    print(data[:-1])


    all_positions = []
    for curve in best_curves:
        position = CurveUtils.generate_positions(curve, body)
        position, measures = CurveGenerator.ray_polygon_colision(body[template], position)
        all_positions.extend(position)
    all_positions = tc.row_stack(all_positions)
    # mesh_manipulation.save_obj("teste.obj", all_positions.cpu().numpy())
    save_mesh = otd.geometry.TriangleMesh()
    save_mesh.vertices = otd.utility.Vector3dVector(all_positions.cpu().numpy())
    otd.io.write_triangle_mesh("teste.obj", save_mesh)

TriangleMesh with 6890 points and 13776 triangles.
     original    measured  error(mm)
0   35.705200   35.704773   0.004272
1  107.182999  107.329384   1.463852
2   99.165901   99.164696   0.012054
3  115.617004  115.025162   5.918427
4   35.556900   35.690845   1.339455
5   65.943703   65.977676   0.339737
TriangleMesh with 27554 points and 55104 triangles.
     original    measured  error(mm)
0   35.705200   35.706299   0.010986
1  107.182999  107.202599   0.195999
2   99.165901   99.156624   0.092773
3  115.617004  115.083244   5.337601
4   35.556900   35.524063   0.328369
5   65.943703   65.831253   1.124496
TriangleMesh with 110210 points and 220416 triangles.
     original    measured  error(mm)
0   35.705200   35.713139   0.079384
1  107.182999  107.203247   0.202484
2   99.165901   99.209145   0.432434
3  115.617004  115.065086   5.519180
4   35.556900   35.514713   0.421867
5   65.943703   66.021690   0.779877


RuntimeError: CUDA out of memory. Tried to allocate 364.00 MiB (GPU 0; 7.93 GiB total capacity; 5.54 GiB already allocated; 295.62 MiB free; 5.63 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF